# File for Prompt Evaluation. 
#### Using Levenshtein Distance, BLEU/ROUGE and XML Validation for Scoring Prompt Candidates

In [ ]:
#!pip3 install levenshtein
#!pip3 install nltk
#!pip3 install rouge-score
#!pip3 install langchain_openai

In [1]:
import os


if not os.environ.get("OPENAI_API_KEY"):
    print("OPEN AI API KEY IS MISSING")

# Initializing ChatGPT/Openai
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
    # base_url="...",
    # organization="...",
    # other params...
)

#### Reading Process Descs

In [2]:
file_process_desc_nebentaetigkeiten = open("assets/process_desc_nebentaetigkeiten.txt", encoding="utf8")
file_process_desc_debriefing = open("assets/process_desc_debriefing.txt", encoding="utf8")
file_process_desc_bedarfsermittlung = open("assets/process_desc_bedarfsermittlung.txt", encoding="utf8")
file_input_example_1 = open("assets/example_for_ai_1.drawio", encoding="utf8")
file_input_example_2 = open("assets/example_for_ai_2.drawio", encoding="utf8")

# save process descriptions
process_desc_nebentaetigkeiten = file_process_desc_nebentaetigkeiten.read()
process_desc_debriefing = file_process_desc_debriefing.read()
process_desc_bedarfsermittlung = file_process_desc_bedarfsermittlung.read()
example_input_file_1 = file_input_example_1
example_input_file_2 = file_input_example_2

# close files
file_process_desc_nebentaetigkeiten.close()
file_process_desc_debriefing.close()
file_process_desc_bedarfsermittlung.close()
file_input_example_1.close()
file_input_example_2.close()

#### Prompts for Evaluation

In [93]:
from langchain_core.prompts import ChatPromptTemplate

# Prompt Extracting Roles
prompt_extracting_roles = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            '''Du bist ein spezialisierter KI-Assistent, dessen einzige Aufgabe es ist, präzise und vollständig alle Rollen zu extrahieren, die in einer bereitgestellten Prozessbeschreibung eine Aktivität ausführen. 
            Rollen und Aktivitäten werden nach der BPMN Spezifikation definiert.
            Antworte ausschließlich mit einer Liste der Rollen im gegebenen Format ohne zusätzliche Erläuterungen oder Informationen.
            Format: ["Eintrag1", "Eintrag2", "Eintrag3"]
            ''',
        ),
        ("human", 
         '''
Extrahiere die Rollen aus der Prozessbeschreibung, indem du die Beschreibung Schritt für Schritt analysierst:

Prozessbeschreibung:
"Mitarbeiter A startet den Prozess und informiert Mitarbeiter B. Mitarbeiter B prüft den Vorgang und leitet ihn an Mitarbeiter C weiter, der schließlich die Freigabe erteilt."
Schritt-für-Schritt Analyse:
Mitarbeiter A handelt → Rolle: Mitarbeiter A
Mitarbeiter B handelt → Rolle: Mitarbeiter B
Mitarbeiter C handelt → Rolle: Mitarbeiter C

Rollen: ["Mitarbeiter A", "Mitarbeiter B", "Mitarbeiter C"]

Prozessbeschreibung:
"Team X erhält die Aufgabe und delegiert sie entweder an Team Y oder Team W. Team Y oder Team W bearbeitet die Aufgabe und sendet die Ergebnisse zur Prüfung an Team Z zurück."
Schritt-für-Schritt Analyse:
Team X erhält und delegiert → Rolle: Team X
Team Y oder W bearbeitet die Aufgabe → Rolle: Team Y, Team W
Team Z prüft die Ergebnisse → Rolle: Team Z

Rollen: ["Team X", "Team Y", "Team W", "Team Z"]

Prozessbeschreibung:
"Person 1 erfasst wichtige Informationen und leitet diese an Person 2 weiter. Person 2 analysiert die Informationen und bespricht eventuelle Rückfragen mit Person 3, bevor die finale Entscheidung getroffen wird."
Schritt-für-Schritt Analyse:
Person 1 handelt → Rolle: Person 1
Person 2 handelt → Rolle: Person 2
Person 3 handelt → Rolle: Person 3

Rollen: ["Person 1", "Person 2", "Person 3"]

Prozessbeschreibung:
"{prozessbeschreibung}"
'''),
    ]
)




# Prompt Extracting Activities
prompt_extracting_activities = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            '''Du bist ein spezialisierter KI-Assistent, dessen einzige Aufgabe es ist, präzise und vollständig alle Aktivitäten zu extrahieren, die in einer bereitgestellten Prozessbeschreibung ausgeführt werden. 
            Aktivitäten werden nach der BPMN Spezifikation definiert.
            Antworte ausschließlich mit einer Liste der Aktivitäten im gegebenen Format ohne zusätzliche Erläuterungen oder Informationen.
            Format: ["Eintrag1", "Eintrag2", "Eintrag3"]''',
        ),
        ("human", 
'''
Du wirst eine Prozessbeschreibung erhalten, aus der du die zugehörigen Aktivitäten extrahieren und diese als Liste in gegebenen Format zurückgeben sollst. 
Im Folgenden siehst du Beispiele für Prozessbeschreibungen und die dazugehörigen Aktivitäten: 
Prozessbeschreibung: “Mitarbeiter A startet den Prozess und informiert Mitarbeiter B. Mitarbeiter B prüft den Vorgang und leitet ihn an Mitarbeiter C weiter, der schließlich die Freigabe erteilt.” 
Aktivitäten: [“Prozess starten”, “Mitarbeiter B informieren”, “Vorgang prüfen”, “an Mitarbeiter C weiterleiten”, “Freigabe erteilen”] 
Prozessbeschreibung: “Team X erhält die Aufgabe und delegiert sie entweder an Team Y oder Team W. Team Y oder Team W bearbeitet die Aufgabe und sendet die Ergebnisse zur Prüfung an Team Z zurück.” 
Aktivitäten: [“Aufgabe an Team Y oder Team W delegieren”, “Aufgabe bearbeiten”, “Ergebnisse zur Prüfung zurück senden”] 
Prozessbeschreibung: “Person 1 erfasst wichtige Informationen und leitet diese an Person 2 weiter. Person 2 analysiert die Informationen und bespricht eventuelle Rückfragen mit Person 3, bevor die finale Entscheidung getroffen wird.” 
Aktivitäten: [“wichtige Informationen erfassen”, “Mitarbeiter B informieren”, “Informationen analysieren”, “eventuelle Rückfragen besprechen”, “Entscheidung treffen”] 
Extrahiere die Aktivitäten aus der folgenden Prozessbeschreibung mithilfe einer schrittweisen Analyse: “{prozessbeschreibung}”
Format: ["Eintrag1", "Eintrag2", "Eintrag3"]
'''),
    ]
)


# Prompt creating Model
prompt_creating_model = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            'Du bist ein Prozessmanager. Dir werden zwei draw.io Dateien als Beispiel gegeben welche ein Prozessmodell darstellen und mit 3 Anführungszeichen abgegrenzt sind. Außerdem werden dir Rollen, Aktivitäten und eine Prozessbeschreibung gegeben. Daraus erstellst du ein Draw.io konformes XML Dokument.',
        ),
        ("human", '''
         Erstelle aus den gegebenen Rollen, Aktivitäten und der Prozessbeschreibung ein Prozessmodell.
         Das XML muss Draw.io konform sein.
         Stelle sicher, dass alle Rollen als Swimlane dargestellt werden.
         Stelle sicher, dass Verzweigungen, signalisiert durch das Wort "oder", als Rauten dargestellt werden und dass diese beschriftet sind.
         Gib mir nur das Ergebnis in XML zurück. 
         Achte vor allem drauf, dass sich keine Elemente überlappen.
         Swimlanes sind immer nebeneinander.
         Achte auf eine angemessen Breite der Swimlanes und verdopple diese dann.
         Die Höhe der Swimlanes muss immer bis an das Ende der Seite gehen.
         Es sollen alle Aktivitäten und Rollen verwendet werden. 
         Rollen die keine Aufgabe übernehmen werden weggelassen.
         
         """{example_input_file_1}"""
         """{example_input_file_2}"""
         
         Rollen: {roles}. 
         Aktivitäten: {activities}. 
         Prozessbeschreibung: {process}
         '''
         ),
    ]
)




#### Initialising Test Cases

In [94]:
class TestCaseExtraction:
    def __init__(self, case_name, process_desc, ground_truth, prompt):
        self.case_name = case_name
        self.process_desc = process_desc
        self.ground_truth = ground_truth
        self.prompt = prompt

class TestCaseModelCreation:
    def __init__(self, case_name, process_desc, input_1, input_2, ground_truth, prompt):
        self.case_name = case_name
        self.input_1 = input_1
        self.input_2 = input_2
        self.process_desc = process_desc
        self.ground_truth = ground_truth
        self.prompt = prompt

# Test Cases for Role Extraction
test_case_roles_bedarfsermittlung =  TestCaseExtraction("Bedarfsermittlung_role_extraction", process_desc_bedarfsermittlung, ["Geschäftsführung", "HR", "Tribe-Lead", "FinCo"], prompt_extracting_roles)
test_case_roles_debriefing = TestCaseExtraction("Debriefing_role_extraction", process_desc_debriefing, ["HR", "Mitarbeiter"], prompt_extracting_roles)
test_case_roles_nebentaetigkeiten = TestCaseExtraction("Nebentaetigkeiten_role_extraction", process_desc_nebentaetigkeiten, ["Mitarbeiter", "HR"], prompt_extracting_roles)

list_test_cases_role_extraction = [test_case_roles_debriefing, test_case_roles_nebentaetigkeiten, test_case_roles_bedarfsermittlung]

# Test Cases for Activity Extraction
activity_list_bedarfsermittlung = ["Bedarfe und Kundenanfragen eruieren", "Prozess unter Staffing dokumentieren", "Bedarf äußern", "Bedarf freigeben", "Stelle ausschreiben", "Bedarf mit Hauptverantwortlichen absprechen"]
activity_list_debriefing = ["Offene Hardware Angelegenheiten klären", "über Projektübergaben sprechen", "Prozess bei Kündigung mitteilen", "Feedback einholen", "Debriefinggespräch führen", "Nach dem Grund der Kündigung fragen", "Offene Themen klären", "Feedback erfragen", "dokumentierte Gespräche ablegen", "Hinweisen oder Missständen nachgehen"]
activity_list_nebentaetigkeiten = ["HR über Nebentätigkeit informieren", "Informationen einholen", "Prüfen ob Nebentätigkeit im Wettbewerb zur Adorsys steht", "Nebentätigkeit ablehnen", "Dokument 'Erlaubnis Nebentätigkeit' erstellen", "Rücksprache mit dem Mitarbeiter oder der Geschäftsführung halten", "Dokument zur Unterschrift vorlegen", "Dokument zusenden", "Dokument einscannen", "Dokument ablegen"]

test_case_activities_bedarfsermittlung = TestCaseExtraction("Bedarfsermittlung_activity_extraction", process_desc_bedarfsermittlung, activity_list_bedarfsermittlung, prompt_extracting_activities)
test_case_activities_debriefing = TestCaseExtraction("Debriefing_activity_extraction", process_desc_debriefing, activity_list_debriefing, prompt_extracting_activities)
test_case_activities_nebentaetigkeiten = TestCaseExtraction("Nebentaetigkeiten_activity_extraction", process_desc_nebentaetigkeiten, activity_list_nebentaetigkeiten, prompt_extracting_activities)

list_test_cases_activity_extraction = [test_case_activities_debriefing, test_case_activities_nebentaetigkeiten, test_case_activities_bedarfsermittlung]

# Test Cases for Model Creation


#### Bleu, Rouge and Meteor Test

## Evaluating Role Extraction Prompt

In [95]:
import Levenshtein as lev
import ast
import pandas as pd
import warnings
import nltk
import nltk.translate.bleu_score
import nltk.translate.meteor_score

from rouge_score import rouge_scorer

#roles_nebentaetigkeiten = []
#roles_nebentaetigkeiten.sort()
#roles_extracted.sort()

#lev.distance(roles_nebentaetigkeiten, roles_extracted)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

df = pd.DataFrame(columns=["Case Name", "Bleu Score", "Rouge 1 Score", "Rouge L Score", "Meteor Score", "System Message", "Prompt", "extracted", "given"])

# init rouge scorer with only 1 grams and RougeL
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

for case in list_test_cases_role_extraction:

    chain = case.prompt | llm
    response_role_extraction = chain.invoke({
        "prozessbeschreibung" : case.process_desc
    })
    
    # Convertion to string list
    extracted_objects = ast.literal_eval(response_role_extraction.content)
    #print(extracted_roles)
    
    extracted_objects.sort()
    case.ground_truth.sort()
    
    # Calc Bleu Score with only 1 grams for roles
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        bleu_score = nltk.translate.bleu_score.sentence_bleu([case.ground_truth], extracted_objects, (1, 0, 0, 0))
    
    # Calc Rouge Score
    scores = scorer.score(str(case.ground_truth), str(extracted_objects))
    rouge1_score = scores["rouge1"].fmeasure
    rougeL_score = scores["rougeL"].fmeasure
    
    # Calc Meteor Score
    meteor_score = nltk.translate.meteor_score.meteor_score([case.ground_truth], extracted_objects)
    
        

    df.loc[len(df)] =(case.case_name, bleu_score, rouge1_score, rougeL_score, meteor_score, case.prompt[0].prompt.template, case.prompt[1].prompt.template, extracted_objects, case.ground_truth)
    
display(df)


,Case Name,Bleu Score,Rouge 1 Score,Rouge L Score,Meteor Score,System Message,Prompt,extracted,given
0,Debriefing_role_extraction,0.367879,0.666667,0.666667,0.263158,"Du bist ein spezialisierter KI-Assistent, dessen einzige Aufgabe es ist, präzise und vollständig alle Rollen zu extrahieren, die in einer bereitgestellten Prozessbeschreibung eine Aktivität ausführen. \n Rollen und Aktivitäten werden nach der BPMN Spezifikation definiert.\n Antworte ausschließlich mit einer Liste der Rollen im gegebenen Format ohne zusätzliche Erläuterungen oder Informationen.\n Format: [""Eintrag1"", ""Eintrag2"", ""Eintrag3""]\n","\nExtrahiere die Rollen aus der Prozessbeschreibung, indem du die Beschreibung Schritt für Schritt analysierst:\n\nProzessbeschreibung:\n""Mitarbeiter A startet den Prozess und informiert Mitarbeiter B. Mitarbeiter B prüft den Vorgang und leitet ihn an Mitarbeiter C weiter, der schließlich die Freigabe erteilt.""\nSchritt-für-Schritt Analyse:\nMitarbeiter A handelt → Rolle: Mitarbeiter A\nMitarbeiter B handelt → Rolle: Mitarbeiter B\nMitarbeiter C handelt → Rolle: Mitarbeiter C\n\nRollen: [""Mitarbeiter A"", ""Mitarbeiter B"", ""Mitarbeiter C""]\n\nProzessbeschreibung:\n""Team X erhält die Aufgabe und delegiert sie entweder an Team Y oder Team W. Team Y oder Team W bearbeitet die Aufgabe und sendet die Ergebnisse zur Prüfung an Team Z zurück.""\nSchritt-für-Schritt Analyse:\nTeam X erhält und delegiert → Rolle: Team X\nTeam Y oder W bearbeitet die Aufgabe → Rolle: Team Y, Team W\nTeam Z prüft die Ergebnisse → Rolle: Team Z\n\nRollen: [""Team X"", ""Team Y"", ""Team W"", ""Team Z""]\n\nProzessbeschreibung:\n""Person 1 erfasst wichtige Informationen und leitet diese an Person 2 weiter. Person 2 analysiert die Informationen und bespricht eventuelle Rückfragen mit Person 3, bevor die finale Entscheidung getroffen wird.""\nSchritt-für-Schritt Analyse:\nPerson 1 handelt → Rolle: Person 1\nPerson 2 handelt → Rolle: Person 2\nPerson 3 handelt → Rolle: Person 3\n\nRollen: [""Person 1"", ""Person 2"", ""Person 3""]\n\nProzessbeschreibung:\n""{prozessbeschreibung}""\n",[HR],"[HR, Mitarbeiter]"
1,Nebentaetigkeiten_role_extraction,0.666667,0.571429,0.571429,0.892857,"Du bist ein spezialisierter KI-Assistent, dessen einzige Aufgabe es ist, präzise und vollständig alle Rollen zu extrahieren, die in einer bereitgestellten Prozessbeschreibung eine Aktivität ausführen. \n Rollen und Aktivitäten werden nach der BPMN Spezifikation definiert.\n Antworte ausschließlich mit einer Liste der Rollen im gegebenen Format ohne zusätzliche Erläuterungen oder Informationen.\n Format: [""Eintrag1"", ""Eintrag2"", ""Eintrag3""]\n","\nExtrahiere die Rollen aus der Prozessbeschreibung, indem du die Beschreibung Schritt für Schritt analysierst:\n\nProzessbeschreibung:\n""Mitarbeiter A startet den Prozess und informiert Mitarbeiter B. Mitarbeiter B prüft den Vorgang und leitet ihn an Mitarbeiter C weiter, der schließlich die Freigabe erteilt.""\nSchritt-für-Schritt Analyse:\nMitarbeiter A handelt → Rolle: Mitarbeiter A\nMitarbeiter B handelt → Rolle: Mitarbeiter B\nMitarbeiter C handelt → Rolle: Mitarbeiter C\n\nRollen: [""Mitarbeiter A"", ""Mitarbeiter B"", ""Mitarbeiter C""]\n\nProzessbeschreibung:\n""Team X erhält die Aufgabe und delegiert sie entweder an Team Y oder Team W. Team Y oder Team W bearbeitet die Aufgabe und sendet die Ergebnisse zur Prüfung an Team Z zurück.""\nSchritt-für-Schritt Analyse:\nTeam X erhält und delegiert → Rolle: Team X\nTeam Y oder W bearbeitet die Aufgabe → Rolle: Team Y, Team W\nTeam Z prüft die Ergebnisse → Rolle: Team Z\n\nRollen: [""Team X"", ""Team Y"", ""Team W"", ""Team Z""]\n\nProzessbeschreibung:\n""Person 1 erfasst wichtige Informationen und leitet diese an Person 2 weiter. Person 2 analysiert die Informationen und bespricht eventuelle Rückfragen mit Person 3, bevor die finale Entscheidung getroffen wird.""\nSchritt-für-Schritt Analyse:\nPerson 1 handelt → Rolle: Person 1\nPerson 2 handelt → Rolle: Person 2\nPers